<a href="https://colab.research.google.com/github/pwilliamspeniel/Deep-Hybrid-Model/blob/main/preprocess_census_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# demographic information of census blocks
df = pd.read_csv('/content/drive/MyDrive/Autoencoders/data_directory/nhgis0011_ds262_20225_blck_grp.csv')

demos_df = df.copy()
demos_df.head()

,GISJOIN,YEAR,STUSAB,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,...,AQRUM034,AQRUM035,AQRUM036,AQRUM037,AQRUM038,AQRUM039,AQRUM040,AQRUM041,AQRUM042,AQRUM043
0,G38000109656001,2018-2022,ND,NaN,NaN,North Dakota,38,Adams County,1,NaN,...,7,10,12,10,19,12,12,10,10,9
1,G38000109656002,2018-2022,ND,NaN,NaN,North Dakota,38,Adams County,1,NaN,...,20,12,32,27,8,19,15,15,12,11
2,G38000309679001,2018-2022,ND,NaN,NaN,North Dakota,38,Barnes County,3,NaN,...,3,18,6,33,6,28,8,9,21,10
3,G38000309679002,2018-2022,ND,NaN,NaN,North Dakota,38,Barnes County,3,NaN,...,28,53,26,23,18,6,7,9,29,15
4,G38000309680001,2018-2022,ND,NaN,NaN,North Dakota,38,Barnes County,3,NaN,...,4,9,10,16,14,4,18,5,9,7


In [2]:
demos_df.describe()

,REGIONA,DIVISIONA,STATEA,COUNTYA,COUSUBA,PLACEA,TRACTA,BLKGRPA,CONCITA,AIANHHA,...,AQRUM034,AQRUM035,AQRUM036,AQRUM037,AQRUM038,AQRUM039,AQRUM040,AQRUM041,AQRUM042,AQRUM043
count,0.0,0.0,632.0,632.000000,0.0,0.0,632.000000,632.000000,0.0,0.0,...,632.000000,632.000000,632.000000,632.000000,632.000000,632.000000,632.000000,632.000000,632.000000,632.000000
mean,NaN,NaN,38.0,49.778481,NaN,NaN,467572.398734,2.064873,NaN,NaN,...,25.060127,25.248418,24.810127,23.704114,23.018987,20.789557,28.905063,23.077532,26.925633,22.169304
std,NaN,NaN,0.0,34.206815,NaN,NaN,473901.779878,1.041409,NaN,NaN,...,30.136607,28.584847,26.695952,25.274270,25.741854,22.342494,31.338404,25.263907,31.378612,25.540648
min,NaN,NaN,38.0,1.000000,NaN,NaN,100.000000,1.000000,NaN,NaN,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
25%,NaN,NaN,38.0,17.000000,NaN,NaN,10600.750000,1.000000,NaN,NaN,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
50%,NaN,NaN,38.0,35.000000,NaN,NaN,40508.000000,2.000000,NaN,NaN,...,14.000000,14.000000,15.000000,14.000000,13.000000,10.000000,18.000000,13.000000,14.000000,10.000000
75%,NaN,NaN,38.0,85.500000,NaN,NaN,961700.000000,3.000000,NaN,NaN,...,28.250000,30.000000,30.000000,27.250000,27.000000,24.000000,34.250000,27.000000,32.000000,26.000000
max,NaN,NaN,38.0,105.000000,NaN,NaN,974200.000000,6.000000,NaN,NaN,...,407.000000,291.000000,311.000000,204.000000,230.000000,168.000000,297.000000,211.000000,324.000000,192.000000


In [3]:
demos_df.isnull().sum()

GISJOIN        0
YEAR           0
STUSAB         0
REGIONA      632
DIVISIONA    632
            ... 
AQRUM039       0
AQRUM040       0
AQRUM041       0
AQRUM042       0
AQRUM043       0
Length: 541, dtype: int64

In [4]:
# prompt: check the number of items

print(f"Number of items: {demos_df.shape[0]}")


Number of items: 632


## Block Group Data

In [5]:
# Define the columns for each age category
age_categories = {
    '25_34yrs': ['AQM4E011', 'AQM4E012', 'AQM4E035', 'AQM4E036'],
    '35_50yrs': ['AQM4E013', 'AQM4E014', 'AQM4E015', 'AQM4E037', 'AQM4E038', 'AQM4E039'],
    'over65yrs': ['AQM4E020', 'AQM4E021', 'AQM4E022', 'AQM4E023', 'AQM4E024', 'AQM4E045', 'AQM4E046', 'AQM4E047', 'AQM4E048', 'AQM4E049']
}

# Calculate the percentages and totals
for category, columns in age_categories.items():
    # Calculate the total count for each age category
    demos_df[category + '_count'] = demos_df[columns].sum(axis=1)
    # Calculate the percentage for each age category
    demos_df['pct' + category] = (demos_df[category + '_count'] / demos_df['AQM4E001']) * 100

# Display the first few rows of the dataframe to check the results
demos_df[['25_34yrs_count', 'pct25_34yrs', '35_50yrs_count', 'pct35_50yrs', 'over65yrs_count', 'pctover65yrs']].head()


,25_34yrs_count,pct25_34yrs,35_50yrs_count,pct35_50yrs,over65yrs_count,pctover65yrs
0,47,6.509695,92,12.742382,172,23.822715
1,148,10.081744,218,14.850136,357,24.318801
2,94,10.633484,74,8.371041,233,26.357466
3,58,6.847698,196,23.140496,146,17.237308
4,58,8.529412,114,16.764706,119,17.500000


In [6]:
# Define the columns for each racial category
racial_categories = {
    'White_alone': ['AQNGE002'],
    'Black_or_African_American_alone': ['AQNGE003'],
    'Other_Race_alone': ['AQNGE004', 'AQNGE005', 'AQNGE006', 'AQNGE007', 'AQNGE008', 'AQNGE009', 'AQNGE010']
}

# Create new columns for each racial category count and percentage
for category, columns in racial_categories.items():
    demos_df[category + '_count'] = demos_df[columns].sum(axis=1)
    demos_df[category + '_pct'] = demos_df[category + '_count'] / demos_df['AQNGE001'] * 100

# Calculate the count for nonwhite category
demos_df['nonwhite_count'] = demos_df[['Black_or_African_American_alone_count', 'Other_Race_alone_count']].sum(axis=1)

# Calculate the percentage for nonwhite category
demos_df['pct_nonwhite'] = demos_df['nonwhite_count'] / demos_df['AQNGE001'] * 100

# Renaming the columns for clarity
demos_df.rename(columns={
    'White_alone_pct': 'pctwhite_alone',
    'Black_or_African_American_alone_pct': 'pctblack_alone',
    'White_alone_count': 'white_alone_count',
    'Black_or_African_American_alone_count': 'black_alone_count',
    'Other_Race_alone_count': 'other_race_count',
    'nonwhite_count': 'nonwhite_count'
}, inplace=True)

# Display the first few rows of the dataframe to check the results
demos_df[['pctwhite_alone', 'pctblack_alone', 'pct_nonwhite', 'white_alone_count', 'nonwhite_count', 'black_alone_count']].head()


,pctwhite_alone,pctblack_alone,pct_nonwhite,white_alone_count,nonwhite_count,black_alone_count
0,97.368421,0.969529,2.631579,703,19,7
1,94.754768,0.272480,7.970027,1391,117,4
2,97.511312,0.000000,4.977376,862,44,0
3,96.458087,0.000000,3.896104,817,33,0
4,96.617647,0.000000,5.735294,657,39,0


In [7]:
# Define columns for educational attainment categories with a degree
edu_columns = ['AQPKE021', 'AQPKE022', 'AQPKE023', 'AQPKE024', 'AQPKE025']

# Calculate total population 25 years and over
total_population_25_over = demos_df['AQPKE001']

# Calculate total population with at least a bachelor's degree
population_with_college_degree = demos_df[edu_columns].sum(axis=1)

# Calculate percentage with a college degree
demos_df['pct_col_degree'] = (population_with_college_degree / total_population_25_over) * 100

# Calculate count of individuals with at least a bachelor's degree
demos_df['col_degree_count'] = population_with_college_degree

# Display the first few rows of the DataFrame to check the results
demos_df[['pct_col_degree', 'col_degree_count']].head()


,pct_col_degree,col_degree_count
0,32.388664,160
1,30.961183,335
2,38.601824,254
3,39.935588,248
4,40.000000,204


In [8]:
# Define columns for travel time to work
travel_time_cols = ['AQN2M002', 'AQN2M003', 'AQN2M004', 'AQN2M005', 'AQN2M006',
                    'AQN2M007', 'AQN2M008', 'AQN2M009', 'AQN2M010']

# Define median values corresponding to each time category
tt_medians = [7.5, 12.5, 17, 22, 27, 32, 39.5, 52, 74.5]

# Calculate total population
total_population = demos_df['AQN2M001']

# Calculate average travel time to work
demos_df['avg_tt_to_work'] = np.dot(demos_df[travel_time_cols].values, tt_medians) / total_population

# Displaying the relevant columns
demos_df[['avg_tt_to_work']].head()


,avg_tt_to_work
0,80.490741
1,62.447917
2,94.635294
3,68.983146
4,64.968354


In [9]:
total_pt_commute = demos_df['AQN2M061']
total_commute = demos_df['AQN2M001']
demos_df['PTcommute_count'] = total_pt_commute

# Calculate percentage of population using public transportation to commute
demos_df['pctPTcommute'] = (total_pt_commute / total_commute) * 100

# Display the percentage and count of population using public transportation to commute
demos_df[['pctPTcommute', 'PTcommute_count']].head()

,pctPTcommute,PTcommute_count
0,18.518519,10
1,10.416667,10
2,11.764706,10
3,11.235955,10
4,12.658228,10


In [10]:
cell_inc = demos_df.groupby(['COUNTYA','TRACTA','BLKGRPA'], as_index=False).apply(lambda x:
                    (x['AQM4E001']*x['AQRAE001']).sum(axis = 0)/x['AQM4E001'].sum(axis=0))

cell_inc.columns = ['COUNTYA', 'TRACTA', 'BLKGRPA', 'inc_per_capita']

demos_df = pd.merge(demos_df, cell_inc, how = 'left', on = ['COUNTYA', 'TRACTA', 'BLKGRPA'])

demos_df[['inc_per_capita']].head()

<ipython-input-10-8ce13e9b702a>:2: RuntimeWarning: invalid value encountered in scalar divide
  (x['AQM4E001']*x['AQRAE001']).sum(axis = 0)/x['AQM4E001'].sum(axis=0))


,inc_per_capita
0,32924.0
1,37259.0
2,39425.0
3,42214.0
4,49606.0


In [11]:
# Selecting specific columns
new_df = demos_df[['COUNTYA', 'TRACTA', 'BLKGRPA','AQM4E001',
                   'pct25_34yrs', 'pct35_50yrs', 'pctover65yrs',
                   'pctwhite_alone', 'pct_nonwhite', 'pctblack_alone',
                   'pct_col_degree', 'pctPTcommute',
                   'white_alone_count', 'nonwhite_count', 'black_alone_count',
                   '25_34yrs_count','35_50yrs_count','over65yrs_count',
                   'col_degree_count', 'PTcommute_count',
                   'avg_tt_to_work', 'inc_per_capita']].fillna(0)


new_df = new_df.rename(columns = {'AQM4E001':'tot_population'})

new_df.to_csv("/content/drive/MyDrive/Autoencoders/demo_blkgrp.csv", index=False)

# Display the first few rows of the new DataFrame
new_df.head()


,COUNTYA,TRACTA,BLKGRPA,tot_population,pct25_34yrs,pct35_50yrs,pctover65yrs,pctwhite_alone,pct_nonwhite,pctblack_alone,...,white_alone_count,nonwhite_count,black_alone_count,25_34yrs_count,35_50yrs_count,over65yrs_count,col_degree_count,PTcommute_count,avg_tt_to_work,inc_per_capita
0,1,965600,1,722,6.509695,12.742382,23.822715,97.368421,2.631579,0.969529,...,703,19,7,47,92,172,160,10,80.490741,32924.0
1,1,965600,2,1468,10.081744,14.850136,24.318801,94.754768,7.970027,0.272480,...,1391,117,4,148,218,357,335,10,62.447917,37259.0
2,3,967900,1,884,10.633484,8.371041,26.357466,97.511312,4.977376,0.000000,...,862,44,0,94,74,233,254,10,94.635294,39425.0
3,3,967900,2,847,6.847698,23.140496,17.237308,96.458087,3.896104,0.000000,...,817,33,0,58,196,146,248,10,68.983146,42214.0
4,3,968000,1,680,8.529412,16.764706,17.500000,96.617647,5.735294,0.000000,...,657,39,0,58,114,119,204,10,64.968354,49606.0


## Tract Level

In [12]:
data_cols = ['tot_population','pct25_34yrs','pct35_50yrs','pctover65yrs',
         'pctwhite_alone','pct_nonwhite','pctblack_alone',
         'pct_col_degree','pctPTcommute','25_34yrs_count','35_50yrs_count','over65yrs_count',
         'white_alone_count','nonwhite_count','black_alone_count',
         'col_degree_count','PTcommute_count','avg_tt_to_work','inc_per_capita']

demos_df = new_df.groupby(['COUNTYA', 'TRACTA'], as_index=False)[data_cols].sum()
demos_df.head()

,COUNTYA,TRACTA,tot_population,pct25_34yrs,pct35_50yrs,pctover65yrs,pctwhite_alone,pct_nonwhite,pctblack_alone,pct_col_degree,...,25_34yrs_count,35_50yrs_count,over65yrs_count,white_alone_count,nonwhite_count,black_alone_count,col_degree_count,PTcommute_count,avg_tt_to_work,inc_per_capita
0,1,965600,2190,16.591439,27.592519,48.141516,192.123189,10.601606,1.242009,63.349847,...,195,310,529,2094,136,11,495,20,142.938657,70183.0
1,3,967900,1731,17.481182,31.511537,43.594774,193.969400,8.873479,0.000000,78.537411,...,152,270,379,1679,77,0,502,20,163.618440,81639.0
2,3,968000,2499,37.687078,44.569389,57.212224,282.337364,31.607072,1.457195,128.736116,...,329,344,497,2326,310,16,877,30,198.356812,151049.0
3,3,968200,2714,41.629547,79.003616,60.510471,365.074401,40.420175,7.897775,153.290926,...,276,542,373,2483,275,54,651,40,232.242996,121900.0
4,3,968300,3882,38.130801,61.448281,93.056328,356.215086,58.763903,12.290090,191.997299,...,373,624,877,3401,650,131,1238,40,215.497970,144218.0


In [13]:
demos_df['geoid'] = '38_'+demos_df['COUNTYA'].astype(str)+'_'+demos_df['TRACTA'].astype(str)
demos_df['geoid'].head()

0    38_1_965600
1    38_3_967900
2    38_3_968000
3    38_3_968200
4    38_3_968300
Name: geoid, dtype: object

In [14]:
demos_df.columns

Index(['COUNTYA', 'TRACTA', 'tot_population', 'pct25_34yrs', 'pct35_50yrs',
       'pctover65yrs', 'pctwhite_alone', 'pct_nonwhite', 'pctblack_alone',
       'pct_col_degree', 'pctPTcommute', '25_34yrs_count', '35_50yrs_count',
       'over65yrs_count', 'white_alone_count', 'nonwhite_count',
       'black_alone_count', 'col_degree_count', 'PTcommute_count',
       'avg_tt_to_work', 'inc_per_capita', 'geoid'],
      dtype='object')

## Census Tract Area

In [15]:
import geopandas as gpd
census_area = gpd.read_file("/content/drive/MyDrive/Autoencoders/data_directory/gis_data/US_tract_2022.shp")

In [16]:
census_area['geometry'] = census_area['geometry'].to_crs('epsg:26916')
census_area['area'] = census_area['geometry'].area/1000000
census_area['geoid'] = census_area['STATEFP'].astype(str)+'_'+census_area['COUNTYFP'].str.lstrip('0')+'_'+census_area['TRACTCE'].str.lstrip('0')

In [17]:
new_df = demos_df.merge(census_area[['geoid','area','INTPTLAT','INTPTLON']], on='geoid')

In [18]:
new_df.to_csv("/content/drive/MyDrive/Autoencoders/demo_tract.csv", index=False)

In [19]:
new_df.columns

Index(['COUNTYA', 'TRACTA', 'tot_population', 'pct25_34yrs', 'pct35_50yrs',
       'pctover65yrs', 'pctwhite_alone', 'pct_nonwhite', 'pctblack_alone',
       'pct_col_degree', 'pctPTcommute', '25_34yrs_count', '35_50yrs_count',
       'over65yrs_count', 'white_alone_count', 'nonwhite_count',
       'black_alone_count', 'col_degree_count', 'PTcommute_count',
       'avg_tt_to_work', 'inc_per_capita', 'geoid', 'area', 'INTPTLAT',
       'INTPTLON'],
      dtype='object')

In [20]:
new_df.head()

,COUNTYA,TRACTA,tot_population,pct25_34yrs,pct35_50yrs,pctover65yrs,pctwhite_alone,pct_nonwhite,pctblack_alone,pct_col_degree,...,nonwhite_count,black_alone_count,col_degree_count,PTcommute_count,avg_tt_to_work,inc_per_capita,geoid,area,INTPTLAT,INTPTLON
0,1,965600,2190,16.591439,27.592519,48.141516,192.123189,10.601606,1.242009,63.349847,...,136,11,495,20,142.938657,70183.0,38_1_965600,2650.221230,+46.0968146,-102.5331983
1,3,967900,1731,17.481182,31.511537,43.594774,193.969400,8.873479,0.000000,78.537411,...,77,0,502,20,163.618440,81639.0,38_3_967900,1643.882426,+46.9370110,-097.8551057
2,3,968000,2499,37.687078,44.569389,57.212224,282.337364,31.607072,1.457195,128.736116,...,310,16,877,30,198.356812,151049.0,38_3_968000,2318.138854,+46.9379265,-098.2386037
3,3,968200,2714,41.629547,79.003616,60.510471,365.074401,40.420175,7.897775,153.290926,...,275,54,651,40,232.242996,121900.0,38_3_968200,15.409636,+46.9195583,-098.0215419
4,3,968300,3882,38.130801,61.448281,93.056328,356.215086,58.763903,12.290090,191.997299,...,650,131,1238,40,215.497970,144218.0,38_3_968300,7.888455,+46.9299130,-097.9842832
